# A Beginner's Introduction to Object-Oriented Programming

This notebook provides a gentle introduction to object-oriented programming (OOP) concepts using financial instruments as examples. By learning OOP through the context of cash flows, bonds, and bank bills, you'll see how OOP concepts apply directly to finance.

## What is Object-Oriented Programming?

Object-Oriented Programming is a programming paradigm based on the concept of 'objects', which are data structures that contain:

1. **Data** (called attributes or properties)
2. **Code** (called methods)

The point of OOP is to build software in a way that reflects the real world.  So think of objects as software representations of things from the real world. A bond is a real world thing that has characteristics (face value, maturity, coupon rate) and behaviors (ways to calculate price and generate cash flows). In OOP, we embed these characteristics and behaviours into software objects that represent and "encapsulate" bonds.

## Classes: The Blueprints for Objects

A class is like a template or blueprint. Just as your university ID card template defines what information every student card must have (student name, ID number, photo, enrollment date), a Bond class defines what information every bond object must have (such as face value, maturity, coupon).

Let's start with a simple `CashFlows` class that models a series of cash flows - the fundamental building block of all financial instruments:

In [1]:
class CashFlows:
    def __init__(self):
        self.maturities = []  # An attribute to store when cash flows occur
        self.amounts = []     # An attribute to store the cash flow amounts
    
    def add_cash_flow(self, maturity, amount):
        """Add a cash flow to the cash flow list."""
        self.maturities.append(maturity)
        self.amounts.append(amount)
    
    def get_cash_flow(self, maturity):
        """Get the cash flow amount for a specific maturity."""
        if maturity in self.maturities:
            return self.amounts[self.maturities.index(maturity)]
        else:
            return None
    
    def get_maturities(self):
        """Return a list of all maturities."""
        return list(self.maturities)
    
    def get_amounts(self):
        """Return a list of all cash flow amounts."""
        return list(self.amounts)
    
    def get_cash_flows(self):
        """Return all cash flows as (maturity, amount) pairs."""
        return list(zip(self.maturities, self.amounts))

Let's break down what's happening in this class:

- `__init__` is a special method (constructor) that initializes a new CashFlows object based on the template (or class)
- `self` refers to the instance of the object being created or manipulated
- `maturities` and `amounts` are attributes that store data about the cash flows
- `add_cash_flow()`, `get_cash_flow()`, `get_maturities()`, etc. are methods that define behaviors

This demonstrates **encapsulation** - each object bundles data (maturities and amounts) with the methods that operate on that data.

## Creating Objects (Instances)

Let's create a cash flow object from our class and add some cash flows to it:

In [2]:
# Create a CashFlows object
my_cash_flows = CashFlows()

# Add some cash flows (e.g., initial investment and future payments)
my_cash_flows.add_cash_flow(0, -100)      # Pay $100 today
my_cash_flows.add_cash_flow(0.5, 5)       # Receive $5 in 6 months
my_cash_flows.add_cash_flow(1.0, 105)     # Receive $105 in 1 year

# Access the data using methods
print("All maturities:", my_cash_flows.get_maturities())
print("All amounts:", my_cash_flows.get_amounts())
print("\nCash flow at time 1.0:", my_cash_flows.get_cash_flow(1.0))
print("\nAll cash flows as pairs:")
for maturity, amount in my_cash_flows.get_cash_flows():
    print(f"  Time {maturity}: ${amount}")

All maturities: [0, 0.5, 1.0]
All amounts: [-100, 5, 105]

Cash flow at time 1.0: 105

All cash flows as pairs:
  Time 0: $-100
  Time 0.5: $5
  Time 1.0: $105


Notice how we interact with the object through its methods rather than directly manipulating the internal data. This is a key principle of **encapsulation** - it protects the internal data and provides a controlled interface.  It also creates a standardised way of interacting with an object regardless of how complex the behaviour might be inside the object.

## Inheritance: Creating Specialized Classes

Inheritance allows us to create new classes based on existing ones. Let's create `Bank_bill` and `Bond` classes that inherit from `CashFlows`. This makes sense because both instruments are fundamentally collections of cash flows, but with specific characteristics and behaviors.

### Bank Bill Class

A bank bill is a zero-coupon instrument - you pay a discounted price today and receive the face value at maturity:

In [3]:
class Bank_bill(CashFlows):  # Bank_bill inherits from CashFlows
    
    def __init__(self, face_value=100, maturity=0.25, ytm=0.00, price=100):
        # Call the parent class's __init__ method
        super().__init__()
        # Add attributes specific to bank bills
        self.face_value = face_value
        self.maturity = maturity
        self.ytm = ytm
        self.price = price
    
    def set_ytm(self, ytm):
        """Set yield to maturity and calculate the corresponding price."""
        self.ytm = ytm
        # Simple interest formula for bank bills
        self.price = self.face_value / (1 + ytm * self.maturity)
    
    def set_price(self, price):
        """Set price and calculate the implied yield to maturity."""
        self.price = price
        # Solve for ytm from the pricing formula
        self.ytm = (self.face_value / price - 1) / self.maturity
    
    def get_price(self):
        return self.price
    
    def get_ytm(self):
        return self.ytm
    
    def set_cash_flows(self):
        """Generate the cash flows for this bank bill."""
        self.add_cash_flow(0, -self.price)
        self.add_cash_flow(self.maturity, self.face_value)

Now let's use our Bank_bill class to create a 3-month bank bill with a 2% yield and interact with it using its methods.

In [4]:
# Create a 3-month bank bill with 2% yield
bill = Bank_bill(face_value=100, maturity=0.25, ytm=0.02)
bill.set_ytm(0.02)  # This calculates the price
bill.set_cash_flows()  # Generate the cash flows

print("Bank Bill:")
print(f"  Face Value: ${bill.face_value}")
print(f"  Maturity: {bill.maturity} years")
print(f"  Yield to Maturity: {bill.get_ytm():.2%}")
print(f"  Price: ${bill.get_price():.2f}")
print(f"  Cash Flows: {bill.get_cash_flows()}")

Bank Bill:
  Face Value: $100
  Maturity: 0.25 years
  Yield to Maturity: 2.00%
  Price: $99.50
  Cash Flows: [(0, -99.50248756218906), (0.25, 100)]


### Bond Class

A bond is more complex - it pays regular coupon payments plus the face value at maturity:

In [5]:
class Bond(CashFlows):  # Bond inherits from CashFlows
    
    def __init__(self, face_value=100, maturity=3, coupon=0.05, frequency=4, ytm=0.05, price=100):
        # Call the parent class's __init__ method
        super().__init__()
        # Add attributes specific to bonds
        self.face_value = face_value
        self.maturity = maturity
        self.coupon = coupon
        self.frequency = frequency  # How many times per year coupons are paid
        self.ytm = ytm
        self.price = price
    
    def set_ytm(self, ytm):
        """Set yield to maturity and calculate the corresponding price using bond pricing formula."""
        self.ytm = ytm
        # Standard bond pricing formula
        coupon_payment = self.face_value * self.coupon / self.frequency
        n_periods = self.maturity * self.frequency
        rate_per_period = ytm / self.frequency
        
        # Present value of coupon payments (annuity)
        pv_coupons = coupon_payment * (1 - (1 + rate_per_period)**(-n_periods)) / rate_per_period
        # Present value of face value
        pv_face = self.face_value / ((1 + rate_per_period)**n_periods)
        
        self.price = pv_coupons + pv_face
    
    def get_price(self):
        return self.price
    
    def get_coupon_rate(self):
        return self.coupon
    
    def get_ytm(self):
        return self.ytm
    
    def set_cash_flows(self):
        """Generate all the cash flows for this bond."""
        # Initial outflow (purchase price)
        self.add_cash_flow(0, -self.price)
        
        # Coupon payments
        coupon_payment = self.face_value * self.coupon / self.frequency
        for i in range(1, self.maturity * self.frequency):
            self.add_cash_flow(i / self.frequency, coupon_payment)
        
        # Final payment (last coupon + face value)
        self.add_cash_flow(self.maturity, self.face_value + coupon_payment)

Just as we did with the bank bill, let's create a specific bond (from the class template) and interact with it.

In [6]:
# Create a 2-year bond with 5% annual coupon, paid quarterly, yielding 4%
bond = Bond(face_value=100, maturity=2, coupon=0.05, frequency=4, ytm=0.04)
bond.set_ytm(0.04)  # This calculates the price
bond.set_cash_flows()  # Generate the cash flows

print("\nBond:")
print(f"  Face Value: ${bond.face_value}")
print(f"  Maturity: {bond.maturity} years")
print(f"  Coupon Rate: {bond.get_coupon_rate():.2%} (paid {bond.frequency} times per year)")
print(f"  Yield to Maturity: {bond.get_ytm():.2%}")
print(f"  Price: ${bond.get_price():.2f}")
print(f"\n  Cash Flows:")
for maturity, amount in bond.get_cash_flows():
    print(f"    Time {maturity:.2f}: ${amount:.2f}")


Bond:
  Face Value: $100
  Maturity: 2 years
  Coupon Rate: 5.00% (paid 4 times per year)
  Yield to Maturity: 4.00%
  Price: $101.91

  Cash Flows:
    Time 0.00: $-101.91
    Time 0.25: $1.25
    Time 0.50: $1.25
    Time 0.75: $1.25
    Time 1.00: $1.25
    Time 1.25: $1.25
    Time 1.50: $1.25
    Time 1.75: $1.25
    Time 2.00: $101.25


Now let's discuss some of the technical software things which are happening in the work we have done so far:
- Both `Bank_bill` and `Bond` inherit from `CashFlows` (indicated by the class name in parentheses).   This means they take the characteristcs and methods of the CashFlows class and build on them, adding further characteristics and methods which are relevant to the specialised nature of bank bills and bonds.   You can imagine that we could further specialse by creating say Bond_Corporate which would inherit from Bond and add a credit spread characteristic.
- Both Bank_bill and Bond call `super().__init__()` to initialize the attributes from the parent class (the maturities and amounts lists).
- Each adds its own unique attributes:
   - `Bank_bill` has: `face_value`, `maturity`, `ytm`, `price`
   - `Bond` also has: `coupon` and `frequency`
- Each implements `set_cash_flows()` differently based on its specific payment structure.   As we discuss below, this is a good example of polymorphism.   We instruct the object to set its cashflows in the same way regardless of whether we are talking to a bond or a bill.   It also illustrates the concept of abstraction: we don't need to know the workings of the internal machinery that goes about calculating the cashflows; we simply trust that the objects know what they are doing inside and get it right!
- Both have methods to calculate price from yield and vice versa.  Again, this shows polymorphism and abstraction.

## Key Observation: Inherited Methods

Notice that both `Bank_bill` and `Bond` can use the methods from `CashFlows` (like `get_cash_flows()`, `get_maturities()`, `get_amounts()`) even though we didn't define them in the child classes. This is the power of **inheritance** - code reuse!

In [7]:
# Both instruments can use methods inherited from CashFlows
print("Bank Bill maturities:", bill.get_maturities())
print("Bond maturities:", bond.get_maturities())

# We can get specific cash flows
print(f"\nBond cash flow at time 0.5: ${bond.get_cash_flow(0.5):.2f}")
print(f"Bank Bill cash flow at maturity: ${bill.get_cash_flow(0.25):.2f}")

Bank Bill maturities: [0, 0.25]
Bond maturities: [0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2]

Bond cash flow at time 0.5: $1.25
Bank Bill cash flow at maturity: $100.00


## Polymorphism: Different Behaviors from the Same Interface

Both `Bank_bill` and `Bond` have a `set_cash_flows()` method, but each implements it differently based on its payment structure. This is called **polymorphism** - the same method name can have different behaviors depending on the object type:

In [8]:
# Create fresh instruments to demonstrate polymorphism
instruments = [
    Bank_bill(face_value=100, maturity=0.5, ytm=0.03),
    Bond(face_value=100, maturity=1, coupon=0.04, frequency=2, ytm=0.04)
]

# Set prices and generate cash flows
for instrument in instruments:
    instrument.set_ytm(0.03)  # Both have this method
    instrument.set_cash_flows()  # Both have this method, but implement it differently!

# Display the results
print("Instrument Cash Flows:\n")
for i, instrument in enumerate(instruments, 1):
    instrument_type = type(instrument).__name__
    print(f"{i}. {instrument_type}:")
    print(f"   Price: ${instrument.get_price():.2f}")
    print(f"   Cash Flows: {instrument.get_cash_flows()}")
    print()

Instrument Cash Flows:

1. Bank_bill:
   Price: $98.52
   Cash Flows: [(0, -98.52216748768474), (0.5, 100)]

2. Bond:
   Price: $100.98
   Cash Flows: [(0, -100.97794171176199), (0.5, 2.0), (1, 102.0)]



Notice how we can treat different types of instruments uniformly in the loop, calling the same methods (`set_ytm()`, `set_cash_flows()`, `get_price()`, `get_cash_flows()`), but each instrument generates its own appropriate cash flow pattern. This is polymorphism in action!

## Key OOP Concepts Demonstrated

Let's review the key OOP concepts we've covered:

1. **Classes and Objects**
   - `CashFlows`, `Bank_bill`, and `Bond` are classes (blueprints)
   - `my_cash_flows`, `bill`, and `bond` are objects (instances)

2. **Attributes**
   - Data stored in objects: `maturities`, `amounts`, `face_value`, `maturity`, `coupon`, `price`, etc.

3. **Methods**
   - Functions attached to objects: `add_cash_flow()`, `get_cash_flows()`, `set_ytm()`, `set_price()`, etc.

4. **Encapsulation**
   - Data and methods are bundled together in classes
   - Internal data is accessed through methods, not directly

5. **Abstraction**
   - Complex functionality is hidden behind simple interfaces
   - Users don't need to know how methods work internally, just how to use them

6. **Inheritance**
   - `Bank_bill` and `Bond` inherit attributes and methods from `CashFlows`
   - Child classes can add new attributes and methods

7. **Polymorphism**
   - `set_cash_flows()` behaves differently depending on the instrument type
   - We can treat different instruments uniformly through their common interface

## Creating collections of objects: Portfolio

Let's create a `Portfolio` class that uses **composition** (containing other objects). A portfolio aggregates multiple instruments:

In [9]:
class Portfolio(CashFlows):
    
    def __init__(self):
        super().__init__()
        self.bonds = []
        self.bank_bills = []
    
    def add_bond(self, bond):
        """Add a bond to the portfolio."""
        self.bonds.append(bond)
        return f"Bond added (maturity: {bond.maturity} years)"
    
    def add_bank_bill(self, bank_bill):
        """Add a bank bill to the portfolio."""
        self.bank_bills.append(bank_bill)
        return f"Bank bill added (maturity: {bank_bill.maturity} years)"
    
    def get_bonds(self):
        return self.bonds
    
    def get_bank_bills(self):
        return self.bank_bills
    
    def set_cash_flows(self):
        """Aggregate all cash flows from all instruments in the portfolio."""
        # Add all cash flows from bonds
        for bond in self.bonds:
            for cash_flow in bond.get_cash_flows():
                self.add_cash_flow(cash_flow[0], cash_flow[1])
        
        # Add all cash flows from bank bills
        for bank_bill in self.bank_bills:
            for cash_flow in bank_bill.get_cash_flows():
                self.add_cash_flow(cash_flow[0], cash_flow[1])
    
    def total_value(self):
        """Calculate the total market value of the portfolio."""
        total = 0
        for bond in self.bonds:
            total += bond.get_price()
        for bill in self.bank_bills:
            total += bill.get_price()
        return total
    
    def list_instruments(self):
        """List all instruments in the portfolio."""
        if not self.bonds and not self.bank_bills:
            return "Portfolio is empty."
        
        result = ["Portfolio Contents:\n"]
        
        if self.bank_bills:
            result.append("Bank Bills:")
            for i, bill in enumerate(self.bank_bills, 1):
                result.append(f"  {i}. Maturity: {bill.maturity} years, "
                            f"Face Value: ${bill.face_value}, Price: ${bill.get_price():.2f}")
        
        if self.bonds:
            result.append("\nBonds:")
            for i, bond in enumerate(self.bonds, 1):
                result.append(f"  {i}. Maturity: {bond.maturity} years, "
                            f"Coupon: {bond.get_coupon_rate():.2%}, Price: ${bond.get_price():.2f}")
        
        result.append(f"\nTotal Portfolio Value: ${self.total_value():.2f}")
        
        return "\n".join(result)

Now let's use our `Portfolio` class:

In [10]:
# Create a portfolio
my_portfolio = Portfolio()

# Create some instruments
bill1 = Bank_bill(face_value=100, maturity=0.25, ytm=0.02)
bill1.set_ytm(0.02)
bill1.set_cash_flows()

bill2 = Bank_bill(face_value=100, maturity=0.5, ytm=0.025)
bill2.set_ytm(0.025)
bill2.set_cash_flows()

bond1 = Bond(face_value=100, maturity=1, coupon=0.04, frequency=2, ytm=0.03)
bond1.set_ytm(0.03)
bond1.set_cash_flows()

bond2 = Bond(face_value=100, maturity=2, coupon=0.05, frequency=4, ytm=0.04)
bond2.set_ytm(0.04)
bond2.set_cash_flows()

# Add instruments to portfolio
print(my_portfolio.add_bank_bill(bill1))
print(my_portfolio.add_bank_bill(bill2))
print(my_portfolio.add_bond(bond1))
print(my_portfolio.add_bond(bond2))

# List all instruments
print("\n" + my_portfolio.list_instruments())

Bank bill added (maturity: 0.25 years)
Bank bill added (maturity: 0.5 years)
Bond added (maturity: 1 years)
Bond added (maturity: 2 years)

Portfolio Contents:

Bank Bills:
  1. Maturity: 0.25 years, Face Value: $100, Price: $99.50
  2. Maturity: 0.5 years, Face Value: $100, Price: $98.77

Bonds:
  1. Maturity: 1 years, Coupon: 4.00%, Price: $100.98
  2. Maturity: 2 years, Coupon: 5.00%, Price: $101.91

Total Portfolio Value: $401.16


In [11]:
# Generate aggregated cash flows for the entire portfolio
my_portfolio.set_cash_flows()

print("\nAggregated Portfolio Cash Flows:")
for maturity, amount in sorted(my_portfolio.get_cash_flows()):
    print(f"  Time {maturity:.2f}: ${amount:.2f}")


Aggregated Portfolio Cash Flows:
  Time 0.00: $-101.91
  Time 0.00: $-100.98
  Time 0.00: $-99.50
  Time 0.00: $-98.77
  Time 0.25: $1.25
  Time 0.25: $100.00
  Time 0.50: $1.25
  Time 0.50: $2.00
  Time 0.50: $100.00
  Time 0.75: $1.25
  Time 1.00: $1.25
  Time 1.00: $102.00
  Time 1.25: $1.25
  Time 1.50: $1.25
  Time 1.75: $1.25
  Time 2.00: $101.25


## Summary: Why OOP is Useful in Finance

Object-Oriented Programming helps us:

1. **Organize code** around financial concepts (bonds, bills, portfolios)

2. **Reuse code** through inheritance (bonds and bills inherit cash flow management)

3. **Hide complexity** through encapsulation (don't need to know the internal pricing formulas)

4. **Work with simple interfaces** through abstraction (interact with objects without understanding their internal implementation)

5. **Model relationships** between entities (portfolios contain instruments)

6. **Build maintainable systems** that can grow (easily add new instrument types like swaps, options, etc.)

7. **Work with uniform interfaces** through polymorphism (all instruments can generate cash flows and calculate prices)

As you practice OOP more, you'll develop an intuition for when and how to create classes to model financial problems. The same principles we used for bonds and bills apply to more complex instruments and systems!